# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# pip install hvplot holoviews

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Convert temperature from Kelvin to Celsius
#city_data_df["Max Temp (C)"] = city_data_df["Max Temp"] - 273.15


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(x= 'Lng', y= 'Lat', size= "Humidity", color= "City", cmap= "Category20", hover_cols=["City", "Humidity"], tiles= 'OSM', geo=True, title='City Humidity Map', xlabel= 'Longitude', ylabel= "Latitude", frame_width=800, frame_height=500, alpha=0.7)    

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[
        (city_data_df["Max Temp"] > 21) & 
        (city_data_df["Max Temp"] < 27) &
        (city_data_df["Wind Speed"] < 4.5) &
        (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df

C:\Users\haave\AppData\Local\Temp\ipykernel_20452\1106999193.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,
573,tatarsk,RU,55.2190,75.9828,89,
574,canchungo,GW,12.0672,-16.0333,61,
575,iquique,CL,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories": "accommodation.hotel",
        "apiKey": geoapify_key,
        "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel_Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Sømandshjemmet
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,faya,SA,18.3851,42.4509,35,No hotel found
1,farsund,NO,58.0948,6.8047,100,Rederiet Hotell
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
3,jamestown,US,42.0970,-79.2353,77,DoubleTree Jamestown
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,Кировская
573,tatarsk,RU,55.2190,75.9828,89,Постоялый двор
574,canchungo,GW,12.0672,-16.0333,61,Hotel Pereira
575,iquique,CL,-20.2208,-70.1431,68,Hostal Vivar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    size="Humidity",
    color="City",
    cmap="Category20",
    hover_cols=["City", "Country", "Humidity",  "Hotel_Name"],
    custom_tooltip=["City", "Country", "Humidity", "Hotel_Name"],  # Add "Hotel_Name" to the tooltip
    tiles='OSM',
    geo=True,
    title='Cities with Ideal Weather Conditions to Travel',
    xlabel='Longitude',
    ylabel="Latitude",
    frame_width=800,
    frame_height=500,
    alpha=0.7
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel_Name)